In [1]:
import requests
import json

url = "https://www.ratemyprofessors.com/graphql"

headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "authorization": "Basic dGVzdDp0ZXN0",
    "cache-control": "no-cache",
    "content-type": "application/json",
    "pragma": "no-cache",
    "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin"
}

payload = {
    "query": "query TeacherSearchPaginationQuery($count: Int!, $cursor: String, $query: TeacherSearchQuery!) { search: newSearch { ...TeacherSearchPagination_search_1jWD3d } } fragment TeacherSearchPagination_search_1jWD3d on newSearch { teachers(query: $query, first: $count, after: $cursor) { didFallback edges { cursor node { ...TeacherCard_teacher id __typename } } pageInfo { hasNextPage endCursor } resultCount filters { field options { value id } } } } fragment TeacherCard_teacher on Teacher { id legacyId avgRating numRatings ...CardFeedback_teacher ...CardSchool_teacher ...CardName_teacher ...TeacherBookmark_teacher } fragment CardFeedback_teacher on Teacher { wouldTakeAgainPercent avgDifficulty } fragment CardSchool_teacher on Teacher { department school { name id } } fragment CardName_teacher on Teacher { firstName lastName } fragment TeacherBookmark_teacher on Teacher { id isSaved }",
    "variables": {
        "count": 10_000,
        "cursor": "YXJyYXljb25uZWN0aW9uOjMx",
        "query": {
            "text": "",
            "schoolID": "U2Nob29sLTExMTI=",
            "fallback": True,
            "departmentID": None
        }
    }
}

response = requests.post(url, headers=headers, data=json.dumps(payload), cookies={'credentials': 'include'})

data = response.json()

In [2]:
edges = data['data']['search']['teachers']['edges']
professor_data = []

for edge in edges:
    professor_data.append({
        'first_name': edge['node']['firstName'],
        'last_name': edge['node']['lastName'],
        'full_name': edge['node']['firstName'] + ' ' + edge['node']['lastName'],
        'avg_rating': edge['node']['avgRating'],
        'rmp_link': f"https://www.ratemyprofessors.com/professor/{edge['node']['legacyId']}",
        'college': 'uiuc',
        'department': edge['node']['department']
    })

In [3]:
print(edges[0])

{'cursor': 'YXJyYXljb25uZWN0aW9uOjMy', 'node': {'__typename': 'Teacher', 'avgDifficulty': 3.7, 'avgRating': 2.7, 'department': 'Economics', 'firstName': 'Bart', 'id': 'VGVhY2hlci0xMjQwMzU=', 'isSaved': False, 'lastName': 'Taub', 'legacyId': 124035, 'numRatings': 13, 'school': {'id': 'U2Nob29sLTExMTI=', 'name': 'University Of Illinois at Urbana - Champaign'}, 'wouldTakeAgainPercent': -1}}


In [4]:
import pymongo
from dotenv import load_dotenv
import os
load_dotenv()

mongodb_uri = os.environ.get("MONGODB_URI")

client = pymongo.MongoClient(mongodb_uri)
db = client[os.environ.get("MONGODB_DB")]
professors = db['professors']

In [5]:
professors.insert_many(professor_data)